In [1]:
import os
import sys
import pathlib
import datetime
import string

ROOT = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(ROOT)

print("ROOT:", ROOT)

ROOT: /data/inr/llm/DIFF_CD/Diffusor


In [2]:
class ARGS:
    def __init__(self):
        self.base_model = "/home/nvidia/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14"
        self.layout_ckpt = os.path.join(ROOT , "outputsV2" , "layout_ddpm_export_80000")
        self.controlnet_ckpt = os.path.join(ROOT , "outputsV2" , "controlnet_ratio_lora_ckpt18000_layout80000")

        self.lora_path = os.path.join(ROOT , "outputsV2" , "lora_loveda_sd15_r8" , "checkpoint-29000")
        self.lora_weight_name = "pytorch_lora_weights.safetensors"
        self.lora_scale = 1.0

        self.image_size = 1024
        self.num_inference_steps_layout = 50
        self.num_inference_steps_image = 50
        self.guidance_scale = 3.5
        self.guidance_rescale = 0.0
        self.control_scale = 2.0  # default in sample_pair.py
        self.seed0 = 123
        self.dtype = "fp16"
        self.device = "cuda"
        self.sampler = "dpmpp_2m"
        self.use_karras_sigmas = False
        self.attention_slicing = "auto"  # set to None to disable
        self.enable_xformers = False
        self.vae_tiling = True
        self.vae_slicing = False
        self.capture_output = False  # True hides live tqdm bars but keeps logs

        self.gpu_ids = [3, 4, 5]

        self.run_id = "1"
        self.save_root = os.path.join(ROOT,"outputsV2" , "resultsgenerator_runs")
        self.outdir = os.path.join(self.save_root, self.run_id)
        
        pathlib.Path(self.outdir).mkdir(parents=True, exist_ok=True)

        self.init_mask = None
        self.mask_format = "loveda_raw"  # loveda_raw or indexed
        self.strength_layout = 0.0


args = ARGS()
print("OUTDIR:", args.outdir)

OUTDIR: /data/inr/llm/DIFF_CD/Diffusor/outputsV2/resultsgenerator_runs/1


In [3]:
# Generate samples + visualize (pure Python, no shell/subprocess)

import re
import json
import argparse
import inspect
from pathlib import Path

# Ensure repo modules are importable in this notebook kernel
DIFFUSERS_SRC = os.path.join(ROOT, "diffusers", "src")
SYNTHETICGEN_ROOT = os.path.join(ROOT, "SyntheticGen")
for p in [DIFFUSERS_SRC, SYNTHETICGEN_ROOT]:
    if p not in sys.path:
        sys.path.insert(0, p)

from src.scripts import sample_pair

# Allow sample_pair.main(args) in notebooks by temporarily overriding parse_args.
if len(inspect.signature(sample_pair.main).parameters) == 0:
    _sample_pair_main = sample_pair.main

    def _main_with_args(parsed_args=None):
        if parsed_args is None:
            return _sample_pair_main()
        original_parse_args = sample_pair.parse_args
        try:
            sample_pair.parse_args = lambda: parsed_args
            return _sample_pair_main()
        finally:
            sample_pair.parse_args = original_parse_args

    sample_pair.main = _main_with_args

# ---- Notebook defaults (edit as needed) ----
args = ARGS()

args.outdir = os.path.join(ROOT, "outputsV2", "results_generator")
args.gpu_ids = []
args.init_mask = None
args.strength_layout = 0.8
args.strength_image = 0.8
args.mask_format = "indexed"
args.ignore_index = 255
args.attention_slicing = False
args.enable_xformers = False
args.vae_tiling = False
args.vae_slicing = False


def _slugify(text: str) -> str:
    text = (text or "").strip().lower()
    text = re.sub(r"[^a-z0-9]+", "-", text)
    return text.strip("-")[:80] or "case"


def _ratios_to_str(ratios):
    if isinstance(ratios, str):
        return ratios
    if isinstance(ratios, dict):
        return ",".join(f"{k}:{v}" for k, v in ratios.items())
    if isinstance(ratios, (list, tuple)):
        return ",".join(str(float(x)) for x in ratios)
    raise TypeError("ratios must be dict, list, or str")


# ---- Define cases (edit these) ----
# You can provide ratios as a list (length == num classes), dict (name->value), or a CSV string.
USER_CASE = {
    "name": "custom",
    "prompt": "a high-resolution satellite image",
    "ratios": [0.05, 0.2, 0.1, 0.05, 0.1, 0.25, 0.25],
    "seed": args.seed0,
}
# Set USER_CASE = None to skip the custom case.
cases = ([USER_CASE] if USER_CASE else []) + [
    {
        "name": "baseline",
        "prompt": "a high-resolution satellite image",
        "ratios": [0.05, 0.2, 0.1, 0.05, 0.1, 0.25, 0.25],
        "seed": args.seed0,
    },
    {
        "name": "water_heavy",
        "prompt": "a high-resolution satellite image, river delta, nadir view",
        "ratios": {"water": 0.45, "road": 0.15, "building": 0.10, "forest": 0.20, "background": 0.10},
        "seed": args.seed0 + 1,
    },
]

# Write config for reproducibility
Path(args.outdir).mkdir(parents=True, exist_ok=True)
(Path(args.outdir) / "cases.json").write_text(json.dumps(cases, indent=2), encoding="utf-8")


def run_case(case: dict, idx: int, gpu_id=None) -> Path:
    name = case.get("name") or f"case_{idx}"
    prompt = case.get("prompt")
    ratios = case.get("ratios")
    seed = int(case.get("seed", args.seed0 + idx))

    out_dir = Path(args.outdir) / f"{idx:03d}_{_slugify(name)}"
    out_dir.mkdir(parents=True, exist_ok=True)

    if (out_dir / "image.png").is_file() and (out_dir / "layout_color.png").is_file():
        print(f"[skip] {out_dir}")
        return out_dir

    device_str = str(args.device)
    if gpu_id is not None and device_str.startswith("cuda"):
        device_str = f"cuda:{int(gpu_id)}"

    sp_args = argparse.Namespace(
        layout_ckpt=str(args.layout_ckpt),
        controlnet_ckpt=str(args.controlnet_ckpt),
        base_model=args.base_model,
        lora_path=str(args.lora_path) if args.lora_path and Path(args.lora_path).exists() else None,
        lora_weight_name=str(args.lora_weight_name),
        lora_scale=float(args.lora_scale),
        save_dir=str(out_dir),
        ratios=_ratios_to_str(ratios),
        ratios_json=None,
        class_names_json=None,
        prompt=str(prompt) if prompt else None,
        init_image=case.get("init_image", None),
        init_mask=case.get("init_mask", args.init_mask),
        strength_layout=float(case.get("strength_layout", args.strength_layout)),
        strength_image=float(case.get("strength_image", args.strength_image)),
        ignore_index=int(case.get("ignore_index", 255)),
        mask_format=str(case.get("mask_format", args.mask_format)),
        seg_ckpt=case.get("seg_ckpt", None),
        seg_arch=str(case.get("seg_arch", "simple")),
        hist_guidance_scale=float(case.get("hist_guidance_scale", 0.0)),
        hist_guidance_temp=float(case.get("hist_guidance_temp", 1.0)),
        guidance_scale=float(case.get("guidance_scale", args.guidance_scale)),
        guidance_rescale=float(case.get("guidance_rescale", args.guidance_rescale)),
        control_scale=float(case.get("control_scale", args.control_scale)),
        attention_slicing=case.get("attention_slicing", args.attention_slicing),
        enable_xformers=bool(case.get("enable_xformers", args.enable_xformers)),
        vae_tiling=bool(case.get("vae_tiling", args.vae_tiling)),
        vae_slicing=bool(case.get("vae_slicing", args.vae_slicing)),
        use_karras_sigmas=bool(case.get("use_karras_sigmas", args.use_karras_sigmas)),
        sampler=str(case.get("sampler", args.sampler)),
        num_inference_steps_layout=int(case.get("num_inference_steps_layout", args.num_inference_steps_layout)),
        num_inference_steps_image=int(case.get("num_inference_steps_image", args.num_inference_steps_image)),
        image_size=int(case.get("image_size", args.image_size)),
        seed=seed,
        dtype=str(case.get("dtype", args.dtype)),
        device=device_str,
    )

    print(f"[run] {idx:03d} {name} -> {out_dir} (device={device_str})")
    sample_pair.main(sp_args)
    return out_dir


# ---- Run ----
results = []
gpu_ids = list(args.gpu_ids) if args.gpu_ids else [None]
for i, case in enumerate(cases):
    results.append(run_case(case, i, gpu_id=gpu_ids[i % len(gpu_ids)]))

print("Done:")
for p in results:
    print(" -", p)


# ---- Visualize ----
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


def _compute_hard_ratios(layout_path: Path, num_classes: int, ignore_index: int = 255):
    arr = np.array(Image.open(layout_path))
    valid = arr != ignore_index
    total = int(valid.sum())
    if total == 0:
        return np.zeros((num_classes,), dtype=np.float32)
    counts = np.bincount(arr[valid].reshape(-1), minlength=num_classes).astype(np.float32)
    return counts / float(total)


def show_result(out_dir: Path):
    meta_path = out_dir / "metadata.json"
    meta = json.loads(meta_path.read_text(encoding="utf-8")) if meta_path.is_file() else {}

    layout_color = Image.open(out_dir / "layout_color.png").convert("RGB")
    image = Image.open(out_dir / "image.png").convert("RGB")

    title = meta.get("prompt", "")
    ratios = meta.get("ratios")
    class_names = meta.get("class_names")

    if class_names:
        num_classes = len(class_names)
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        axs[0].imshow(layout_color)
        axs[0].set_title("layout_color.png")
        axs[0].axis("off")
        axs[1].imshow(image)
        axs[1].set_title("image.png")
        axs[1].axis("off")

        r_hat = _compute_hard_ratios(out_dir / "layout.png", num_classes=num_classes)
        x = np.arange(num_classes)
        if ratios is not None:
            axs[2].bar(x - 0.2, ratios, width=0.4, label="target")
        axs[2].bar(x + 0.2, r_hat, width=0.4, label="hard")
        axs[2].set_ylim(0.0, 1.0)
        axs[2].set_title("ratios")
        axs[2].set_xticks(x)
        axs[2].set_xticklabels(class_names, rotation=45, ha="right")
        axs[2].legend()
    else:
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        axs[0].imshow(layout_color)
        axs[0].set_title("layout_color.png")
        axs[0].axis("off")
        axs[1].imshow(image)
        axs[1].set_title("image.png")
        axs[1].axis("off")

    fig.suptitle(title)
    plt.show()


for out_dir in results:
    show_result(Path(out_dir))


/home/nvidia/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.6.2) or chardet (None)/charset_normalizer (3.4.4) doesn't match a supported version!
  warnings.warn(
/home/nvidia/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 